In [1]:
import pandas as pd
import backtrader as bt
import datetime

Since we need OHLC data for backtesting in backtrader, lets convert tick data to OHLC data using Pandas resample method.
Let's set the timeframe as 100 milliseconds

In [2]:
#loading the data
df = pd.read_csv('BTCUSDT-trades-2022-03_price.csv', parse_dates=True, index_col=0)
df.head()

,price
dt,
2022-03-01 00:00:00.000,43160.00
2022-03-01 00:00:00.003,43160.01
2022-03-01 00:00:00.004,43160.01
2022-03-01 00:00:00.005,43160.01
2022-03-01 00:00:00.006,43160.00


In [3]:
# Convert to OHLC
time_frame = '100ms'
df.index = pd.to_datetime(df.index)
df_ohlc = df['price'].resample(time_frame).ohlc()
df_ohlc

,open,high,low,close
dt,,,,
2022-03-01 00:00:00.000,43160.00,43160.01,43155.68,43159.96
2022-03-01 00:00:00.100,43159.96,43160.23,43159.96,43160.23
2022-03-01 00:00:00.200,43160.23,43161.40,43160.23,43161.40
2022-03-01 00:00:00.300,43163.72,43168.04,43163.72,43168.04
2022-03-01 00:00:00.400,NaN,NaN,NaN,NaN
...,...,...,...,...
2022-03-03 23:59:59.500,NaN,NaN,NaN,NaN
2022-03-03 23:59:59.600,NaN,NaN,NaN,NaN
2022-03-03 23:59:59.700,NaN,NaN,NaN,NaN


Let's look for missing value

In [4]:
df_ohlc.isnull().sum()

open     1432463
high     1432463
low      1432463
close    1432463
dtype: int64

Looking at the missing data, it looks like no trades had happened in those 100 ms window where we have missing values.
We can either fill the rows with missing values with closing price of the previous row OR just ignore the missing values by deleting them.
Since both of the above option would yield similar results it would be wiser to drop the missing rows for faster computation

In [5]:
df_ohlc.dropna(inplace=True)
df_ohlc

,open,high,low,close
dt,,,,
2022-03-01 00:00:00.000,43160.00,43160.01,43155.68,43159.96
2022-03-01 00:00:00.100,43159.96,43160.23,43159.96,43160.23
2022-03-01 00:00:00.200,43160.23,43161.40,43160.23,43161.40
2022-03-01 00:00:00.300,43163.72,43168.04,43163.72,43168.04
2022-03-01 00:00:00.500,43166.99,43166.99,43166.98,43166.98
...,...,...,...,...
2022-03-03 23:59:55.800,42454.00,42454.00,42454.00,42454.00
2022-03-03 23:59:57.700,42454.00,42454.00,42454.00,42454.00
2022-03-03 23:59:59.100,42454.01,42454.01,42454.01,42454.01


Let's save this data to a csv file

In [6]:
df_ohlc.to_csv('BTCUSDT-trades-2022-03_price_ohlc.csv')

Market Making strategy:
    * Place bid and ask order 3 basis points away from ltp
    * each order value is 1000 USD
    * once any order is filled place another bid and ask order 3 basis points away
    * If position exceeds 5000 usd on any long side place one sell order 0.01 points below ltp
    * If position exceeds 5000 usd on any short side place one buy order 0.01 points above ltp
    * Broker commission is zero

In [49]:
class CommInfoFractional(bt.CommissionInfo):
    def getsize(self, price, cash):
        '''Returns fractional size for cash operation @price'''
        return self.p.leverage * (cash / price)

class TestStrategy(bt.Strategy):

    def log(self, txt):
        ''' Logging function for this strategy'''
        dt = self.datas[0].datetime.datetime(0)
        print(dt, txt)

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        #keep a reference to net long/short position
        self.net_position = 0

        #flag to place bid and ask orders only after an order is executed
        self.place_bid_and_offer_order = True


    def notify_order(self, order):

        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            self.place_bid_and_offer_order = True
            if order.isbuy():
                self.net_position += 1
                self.log(str(order.executed.price) + ' qty:' + str(order.executed.size))
            elif order.issell():
                self.net_position -= 1
                self.log(str(order.executed.price) + ' qty:' + str(order.executed.size))

            print("net position is", self.net_position)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')


    def next(self):

        if self.place_bid_and_offer_order:
            self.place_bid_and_offer_order = False

            if self.net_position in range(-4, 5):
                #Place limit order bid at 3 basis points below ltp
                bid_price = round(self.dataclose[0] - (self.dataclose[0] * 0.0003), 2)
                #set trade value as $1000
                bid_qty = 1000/bid_price
                self.order = self.buy(exectype=bt.Order.Limit, price= bid_price, size = bid_qty)
            
                #Place limit sell order at 3 basis points above ltp
                ask_price = round(self.dataclose[0] + (self.dataclose[0] * 0.0003),2)
                #set trade value as $1000
                ask_qty = 1000/ask_price
                self.order = self.sell(exectype=bt.Order.Limit, price= ask_price, size = ask_qty)


            if self.net_position <= -5:
                #Place limit order bid at 0.05 above ltp to neutralise position
                bid_price = self.dataclose[0] + 0.01
                bid_qty = 1000/bid_price
                self.order = self.buy(exectype=bt.Order.Limit, price= bid_price, size = bid_qty)
            
            if self.net_position >= 5:
                #Place limit sell order 0.05 points below ltp to neutralise position
                ask_price = self.dataclose[0] - 0.01
                ask_qty = 1000/ask_price
                self.order = self.sell(exectype=bt.Order.Limit, price= ask_price, size = ask_qty)

        else:
            return


cerebro = bt.Cerebro()

cerebro.addstrategy(TestStrategy)

#load the OHLC data for backtesting
data = bt.feeds.GenericCSVData(dataname='BTCUSDT-trades-2022-03_price_ohlc.csv', dtformat=('%Y-%m-%d %H:%M:%S.%f'), datetime=0, open=1, high=2, low=3, close=4,
    volume=-1, openinterest=-1, timeframe=bt.TimeFrame.MicroSeconds, compression=100000)

cerebro.adddata(data)

#add the required analyzers
cerebro.addanalyzer(bt.analyzers.Transactions, _name='tradess')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades_statss')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown_statss')

cerebro.broker.setcash(1000000.0)

# Set the commission
cerebro.broker.addcommissioninfo(CommInfoFractional())
cerebro.broker.setcommission(commission=0.0)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

strats = cerebro.run(maxcpus=4)

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 1000000.00
2022-03-01 00:00:01.200003 43173.65 qty:-0.023162673074388546
net position is -1
2022-03-01 00:00:03.100003 43187.25 qty:-0.023154982083832612
net position is -2
2022-03-01 00:00:03.200002 43185.62 qty:0.02315510003813645
net position is -1
2022-03-01 00:00:03.799999 43171.73 qty:0.023163306172812623
net position is 0
2022-03-01 00:00:04.599995 43212.95 qty:-0.023141211141567516
net position is -1
2022-03-01 00:00:04.599995 43197.65 qty:-0.023149407433043234
net position is -2
2022-03-01 00:00:04.599995 43184.34 qty:-0.023156542394766253
net position is -3
2022-03-01 00:00:04.700004 43180.39 qty:0.02314067028339916
net position is -2
2022-03-01 00:00:09.599998 43206.55 qty:-0.023147280125143454
net position is -3
2022-03-01 00:00:10 43193.59 qty:0.023151583371514156
net position is -2
2022-03-01 00:00:13.699996 43208.6 qty:-0.023145083567638728
net position is -3
2022-03-01 00:00:44.400000 43195.64 qty:0.023150484632245293
net position is -2
2022-03

In [ ]:
#All the trades executed are stored in transactions_df
transactions_list = []
for trade_dt, trade_values in strats[0].analyzers.tradess.get_analysis().items():
    transactions_list.append([trade_dt, trade_values[0][1], trade_values[0][0], -trade_values[0][4]])
transactions_df = pd.DataFrame(transactions_list, columns = ['datetime', 'price', 'quantity', 'trade_value'])
transactions_df

Assignment III - Execution Analysis

In [ ]:
results = strats[0].analyzers.trades_statss.get_analysis()
number_of_trades=results['total']['total']
gross_pnl = results['pnl']['gross']['total']
gross_pnl_in_basis_points = gross_pnl/(number_of_trades*1000)*10000
average_win_pnl = results['won']['pnl']['average']
average_loss_pnl = results['lost']['pnl']['average']

In [ ]:
max_drawdown = strats[0].analyzers.drawdown_statss.get_analysis()['max']['moneydown']

In [ ]:
print("1) Gross PNL in basis points:", gross_pnl_in_basis_points)
print("2) Gross PNL in $:", gross_pnl)
print("3) Maximum drawdown:", max_drawdown)
print("4) Average win pnl:", average_win_pnl)
print("5) Average loss pnl:", average_loss_pnl)
print("6) Number of trades:", number_of_trades)